# Transform ASK library

In [1]:
from datasets import load_dataset

dataset = load_dataset("ctu-aic/ask_library_cs", split="train")
dataset

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id', 'language', 'question_diacritics_korektor', 'answer_diacritics_korektor', 'question_spell_korektor', 'answer_spell_korektor', 'diacritics_correct'],
    num_rows: 16273
})

## Korektor

In [10]:
!/home/mlynatom/korektor/korektor-2.0.0-bin/bin-linux64/korektor

Usage: /home/mlynatom/korektor/korektor-2.0.0-bin/bin-linux64/korektor [options] model_configuration
Options: --input=untokenized|untokenized_lines|segmented|vertical|horizontal
         --output=original|xml|vertical|horizontal
         --corrections=maximum_number_of_corrections
         --viterbi_order=viterbi_decoding_order
         --viterbi_beam_size=maximum_viterbi_beam_size
         --viterbi_stage_pruning=maximum_viterbi_stage_cost_increment
         --context_free
         --version
         --help


In [ ]:
import subprocess


command = "echo 'cau jak se mas?' | /home/mlynatom/korektor/korektor-2.0.0-bin/bin-linux64/korektor /home/mlynatom/korektor/korektor-czech-130202/diacritics_h2mor.conf"

result = subprocess.run(command, shell=True, text=True, capture_output=True)

print(result.stdout.strip())

čau jak se máš?


In [14]:
#korektor
import requests
import subprocess

korektor_url = "https://lindat.mff.cuni.cz/services/korektor/api/correct"
url_model_diacritics = 'czech-diacritics_generator-130202'
url_model_spell = 'czech-spellchecker-130202'

korektor_path = "/home/mlynatom/korektor/korektor-2.0.0-bin/bin-linux64/korektor"
path_model_diacritics = "/home/mlynatom/korektor/korektor-czech-130202/diacritics_h2mor.conf"
path_model_spell = "/home/mlynatom/korektor/korektor-czech-130202/spellchecking_h2mor.conf"


def correct_text(text, model, korektor_url=None, korektor_path=None):
    if korektor_url is None and korektor_path is None:
        raise ValueError("Either korektor_url or korektor_path must be provided")
    elif korektor_url is not None:
        params = {
            'model': model,
            'data': text,
        }
        response = requests.post(korektor_url, data=params)
        if response.status_code != 200:
            raise Exception(f"Failed to correct text: {response.text}")
        corrected_text = response.json()['result']
    else:
        command = f"echo '{text}' | {korektor_path} {model}"
        result = subprocess.run(command, shell=True, text=True, capture_output=True)
        corrected_text = result.stdout.strip()
    return corrected_text



In [15]:
correct_text("Toto je testovací veta bez diakritiky.", korektor_path = korektor_path, model = path_model_diacritics)

'Toto je testovací věta bez diakritiky.'

In [16]:
def correct_map(example):
    if example['language'] == 'cs':
        example['question_diacritics_korektor'] = correct_text(example['question'],korektor_path = korektor_path, model = path_model_diacritics)
        example['answer_diacritics_korektor'] = correct_text(example['answer'],korektor_path = korektor_path, model = path_model_diacritics)
        example['question_spell_korektor'] = correct_text(example['question'], korektor_path = korektor_path, model = path_model_spell)
        example['answer_spell_korektor'] = correct_text(example['answer'], korektor_path = korektor_path, model = path_model_spell)
    else:
        example['question_diacritics_korektor'] = ""
        example['answer_diacritics_korektor'] = ""
        example['question_spell_korektor'] = ""
        example['answer_spell_korektor'] = ""
    return example

In [17]:
dataset = dataset.map(correct_map)

Map:   0%|          | 0/16273 [00:00<?, ? examples/s]

In [18]:
dataset

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id', 'language', 'question_diacritics_korektor', 'answer_diacritics_korektor', 'question_spell_korektor', 'answer_spell_korektor'],
    num_rows: 16273
})

In [25]:
from huggingface_hub import login
login()

In [26]:
dataset.push_to_hub("ctu-aic/ask_library_cs", commit_message="Columns with corrected text using korektor")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/17 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ctu-aic/ask_library_cs/commit/df8ccc4930f65e92cfbb11eb56195e545ac6e36a', commit_message='Columns with corrected text using korektor', commit_description='', oid='df8ccc4930f65e92cfbb11eb56195e545ac6e36a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ctu-aic/ask_library_cs', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ctu-aic/ask_library_cs'), pr_revision=None, pr_num=None)

## Other

### Diacritics

In [2]:
q = dataset['question']
diacritic_q = dataset['question_diacritics_korektor']
a = dataset['answer']
diacritic_a = dataset['answer_diacritics_korektor']

import nltk
from tqdm import tqdm

diacritics_correct = []

for q, dq, a, da in tqdm(zip(q, diacritic_q, a, diacritic_a)):
    threshold_q = len(q)*0.01
    threshold_a = len(a)*0.01
    if len(dq) > 0 and len(da) > 0 and q!=dq and a!=da and (nltk.edit_distance(q, dq) > threshold_q or nltk.edit_distance(a, da) > threshold_a):
        diacritics_correct.append(False)
    else:
        diacritics_correct.append(True)

diacritics_correct

16273it [3:34:56,  1.26it/s]


[True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [9]:
dataset = dataset.remove_columns(["diacritics_correct"])

In [10]:
dataset = dataset.add_column('diacritics_correct', diacritics_correct)

In [11]:
dataset[3]

{'title': '1. cesky preklad Cervantesova dona Qiujota',
 'question': 'Zajima me, za jakeho duvodu se kun dona Quijota objevuje v ceskych prekladech a nasledkem toho i v ceskem povedomi (a v bas. motivech) jako KOBYLA Rosinanta, kdyz v originale je to hrebec Rocinante. DQ ho v knize pojmenovana vcetne vyslevetleni motivace tohoto pojmenovani a krome toho je v knize scena, kdy Rocinante potka jinou kobylu - toto setkani je v Cj dost komicke. Chtel bych proto nalezt prvni preklady romanu o DQ to CJ, priapdne jeho casti, pripadne preklady do NJ, ktere se objevovaly na uzemi Cech v 19. stol (nebo i drive? - tezko). Pokud je prvnim prekladem vydani Josefa Pečírky z roku 1864 (gen. kat. I, Cer 431), jak se tam Rocinante objevuje? A Pichluv preklad u Kobra 1866,1868? Pripadne pokud nekdo tento \\"problem\\" vyresil - kdo a kde? Velice dekuji za odpoved a doufam ze neni prilis narocna...',
 'answer': 'Jmeno kone je ve dvou prvnich ceskych vydanich (1864, 1866) uvedeno v nasledujici forme: Blazn

In [12]:
dataset.push_to_hub("ctu-aic/ask_library_cs", commit_message="Added column with diacritics correction evaluation")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/17 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ctu-aic/ask_library_cs/commit/82b5a6aefb52af2f125dc1637001427ce66d0dca', commit_message='Added column with diacritics correction evaluation', commit_description='', oid='82b5a6aefb52af2f125dc1637001427ce66d0dca', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ctu-aic/ask_library_cs', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ctu-aic/ask_library_cs'), pr_revision=None, pr_num=None)

### Filter with correct diacritics

In [2]:
dataset = dataset.filter(lambda x: x['diacritics_correct'])
dataset

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id', 'language', 'question_diacritics_korektor', 'answer_diacritics_korektor', 'question_spell_korektor', 'answer_spell_korektor', 'diacritics_correct'],
    num_rows: 14659
})

### Filter Czech Rows

In [3]:
#filter czech
dataset = dataset.filter(lambda example: example['language'] == 'cs')
dataset

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id', 'language', 'question_diacritics_korektor', 'answer_diacritics_korektor', 'question_spell_korektor', 'answer_spell_korektor', 'diacritics_correct'],
    num_rows: 14175
})

### Take care of whitespace and newline chars

In [4]:
#filter all newline and whitespace characters
import re

def clean_text(example):
    #match all whitespace characters and replace them with a single space
    example['question'] = re.sub(r'\s+', ' ', example['question']).strip()
    example['answer'] = re.sub(r'\s+', ' ', example['answer']).strip()
    example['question'] = re.sub(r'\xa0', ' ', example['question']).strip()
    example['answer'] = re.sub(r'\xa0', ' ', example['answer']).strip()
    return example

dataset = dataset.map(clean_text)

Map:   0%|          | 0/14175 [00:00<?, ? examples/s]

In [5]:
dataset[0]

{'title': 'e-knihy',
 'question': 'Dobrý den, nemohu se prosím přihlásit přes zadané údaje, hlásí to chybně zadaný pin nebo čtenářské číslo, delší období jsem knihy nevypůjčovala, chci teď v době koronaviru zkusit e-lerning. Děkuji za odpověď.',
 'answer': 'Vážená paní, moc mě to mrzí, ale v databázi čtenářů Vás již nemáme. Podle Nařízení na ochranu osobních údajů nemůžeme spravovat údaje čtenářů déle než šest měsíců od propadlé registrace. Potom musíme všechny Vaše údaje zarchivovat. Nabídnout Vám mohu založení nového čtenářského konta, toto lze však učinit až v okamžiku znovuotevření knihovny. Momentálně můžete zkusit e-knihy, které jdou stahovat zdarma, více informací získáte na tomto webu -https://protiviru.knihovny.cz/ Děkuji za pochopení a přeji pevné zdraví',
 'okres': 'Hradec Králové',
 'library': 'Knihovna města Hradce Králové',
 'category': 'Informace o knihovně',
 'used_sources': None,
 'date': Timestamp('2020-03-23 08:00:00'),
 'url': 'https://www.ptejteseknihovny.cz/dotazy

### Filter URLS

In [6]:
import re
import nltk
nltk.download('punkt_tab')

def remove_url(text:str) -> str:
    url_in_brackets_pattern = re.compile(r'\s*\([^)(]*https?:\/\/[^\s()]+[^)(]*\)')
    www_in_brackets_pattern = re.compile(r'\s*\([^)(]*www.[^\s()]+[^)(]*\)')
    if "http" in text or "www" in text:
        #remove urls in brackets
        text = re.sub(url_in_brackets_pattern, '', text)
        text = re.sub(www_in_brackets_pattern, '', text)

        #if url still in the answer
        if "http" in text or "www" in text:
            #handle standalone urls
            ##obtain sentences
            sentences=nltk.tokenize.sent_tokenize(text, language="czech")
            ##find sentences without urls
            filtered_sentences = [sentence for sentence in sentences if "http" not in sentence and "www" not in sentence]
            
            text = " ".join(filtered_sentences)

    
    return text

dataset = dataset.map(lambda x: {'question': remove_url(x['question']), 'answer': remove_url(x['answer'])})
dataset

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/mlynatom/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Map:   0%|          | 0/14175 [00:00<?, ? examples/s]

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id', 'language', 'question_diacritics_korektor', 'answer_diacritics_korektor', 'question_spell_korektor', 'answer_spell_korektor', 'diacritics_correct'],
    num_rows: 14175
})

### Dobry den - filter

In [7]:
dataset = dataset.filter(lambda x: "Dobry den" not in x["answer"] and "dobry den" not in x["answer"] and "Dobry den" not in x["question"] and "dobry den" not in x["question"] and "Vazena pani" not in x["answer"] and "vazena pani" not in x["answer"] and "Vazena pani" not in x["question"] and "vazena pani" not in x["question"] and "Vazeny pane" not in x["answer"] and "vazeny pane" not in x["answer"] and "Vazeny pane" not in x["question"] and "vazeny pane" not in x["question"])
dataset

Filter:   0%|          | 0/14175 [00:00<?, ? examples/s]

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id', 'language', 'question_diacritics_korektor', 'answer_diacritics_korektor', 'question_spell_korektor', 'answer_spell_korektor', 'diacritics_correct'],
    num_rows: 13663
})

### Filter e-mails

In [8]:
import re

def remove_url(text:str) -> str:
    email_in_brackets_pattern = re.compile(r'\s*\([^)(]*[a-zA-Z0-9\.]+@+[a-zA-Z0-9]+\.[a-zA-Z0-9]+[^)(]*\)')
    text = re.sub(email_in_brackets_pattern, '', text)
    
    return text

dataset = dataset.map(lambda x: {'question': remove_url(x['question']), 'answer': remove_url(x['answer'])})
dataset

Map:   0%|          | 0/13663 [00:00<?, ? examples/s]

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id', 'language', 'question_diacritics_korektor', 'answer_diacritics_korektor', 'question_spell_korektor', 'answer_spell_korektor', 'diacritics_correct'],
    num_rows: 13663
})

### Filter greetings

In [9]:
FILTER_GREETING = ["Vážený paní", "Dobrý den", "Vážený pane", "Vážený čtenáři", "Vážená paní", "Dobré odpoledne", "Hezký den", "Dobrá den", "Dobrý de", "Dobré ráno", "Dobrý večer", "dobrý den", "Vážný pane", "Váženy pane", "Važený pane", "Dobrý dne", "DObrý den", "Dobré dopoledne"]

#find greeting using along with the name and other info up to first , -> remove it and change the next letter to uppercase
greetings_regexes = [re.compile(fr"^{greeting}[^,]*,\s*", re.IGNORECASE) for greeting in FILTER_GREETING]

def remove_greetings(example):
    answer = example['answer']
    for greeting_regex in greetings_regexes:
        answer = re.sub(greeting_regex, '', answer)
        
    #first letter to uppercase
    if len(answer) > 0:
        answer = answer[0].upper() + answer[1:]
    example['answer'] = answer
    return example

dataset = dataset.map(remove_greetings)
dataset

Map:   0%|          | 0/13663 [00:00<?, ? examples/s]

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id', 'language', 'question_diacritics_korektor', 'answer_diacritics_korektor', 'question_spell_korektor', 'answer_spell_korektor', 'diacritics_correct'],
    num_rows: 13663
})

In [13]:
dataset[1234]

{'title': 'Činžák',
 'question': 'Kdy se začali stavět činžovní domy a z jaké doby zhruba je tento výraz?Kdo přišel s tímto způsobem bydlení a kde to vzniklo?Děkuji',
 'answer': 'Vícepodlažní nájemné domy začaly být budovány již ve starověkém Římě. Nazývaly se insula (ostrov) a mohly dosahovat výšky až 8 - 9 pater. Domy "měly často do ulice honosné průčelí se sloupovím, ale uvnitř bylo namačkáno množství malých bytů, často s místnostmi v několika řadách za sebou, takže ty vnitřní neměly denní světlo. Horní patra byla často hrázděná (dřevěná sloupková konstrukce s cihlovými nebo hliněnými výplněmi) nebo dřevěná a zřejmě v nich docházelo k častým zřícením a požárům. Proto Augustus omezil výšku insulí na 70 stop (asi 20,75 m) a později byla dál snížena na 60 stop (17, 55 m). I tak se do stanovené výšky vešlo 5 - 7 podlaží. V té době bylo v Římě 1790 domů s atriem, zvaných domus (dům), a celých 44 602 nájemných insulí, každá s několika desítkami obytných místností" (Zdroj: SYROVÝ, Petr. Do

### final map

In [15]:
from ftfy import fix_text


def knihovna_map(example):
    input = example["question"]
    input = fix_text(input.strip())
    output = fix_text(example["answer"].strip())

    if input == "" or output == "":
        example["conversations"] = None
        return example
    
    user_conversation = {"role": "user", "content": input}
    assistant_conversation = {"role": "assistant", "content": output}

    conversations = [
        user_conversation, assistant_conversation
    ]

    example["conversations"] = conversations

    return example


dataset = dataset.map(knihovna_map)

Map:   0%|          | 0/13663 [00:00<?, ? examples/s]

In [17]:
dataset[0]

{'title': 'e-knihy',
 'question': 'Dobrý den, nemohu se prosím přihlásit přes zadané údaje, hlásí to chybně zadaný pin nebo čtenářské číslo, delší období jsem knihy nevypůjčovala, chci teď v době koronaviru zkusit e-lerning. Děkuji za odpověď.',
 'answer': 'Moc mě to mrzí, ale v databázi čtenářů Vás již nemáme. Podle Nařízení na ochranu osobních údajů nemůžeme spravovat údaje čtenářů déle než šest měsíců od propadlé registrace. Potom musíme všechny Vaše údaje zarchivovat. Nabídnout Vám mohu založení nového čtenářského konta, toto lze však učinit až v okamžiku znovuotevření knihovny.',
 'okres': 'Hradec Králové',
 'library': 'Knihovna města Hradce Králové',
 'category': 'Informace o knihovně',
 'used_sources': None,
 'date': Timestamp('2020-03-23 08:00:00'),
 'url': 'https://www.ptejteseknihovny.cz/dotazy/-3',
 'id': 0,
 'language': 'cs',
 'question_diacritics_korektor': 'Dobrý den, nemohu se prosím přihlásit přes zadané údaje, hlásí to chybně žádaný pin nebo čtenářské číslo, delší obdo

In [18]:
dataset = dataset.filter(lambda x: x['conversations'] is not None)
dataset

Filter:   0%|          | 0/13663 [00:00<?, ? examples/s]

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id', 'language', 'question_diacritics_korektor', 'answer_diacritics_korektor', 'question_spell_korektor', 'answer_spell_korektor', 'diacritics_correct', 'conversations'],
    num_rows: 13374
})

In [19]:
dataset.save_to_disk("/home/mlynatom/master-thesis-repository-tomas-mlynar/datasets/instruction_tuning/ask_library/ask_library_cs_itformat")

Saving the dataset (0/1 shards):   0%|          | 0/13374 [00:00<?, ? examples/s]